In [ ]:
import pickle
import sys
from os import path
from time import time
from warnings import warn

import numpy as np
from divergence import FisherDivergence, KnnKS, ModelKS
from featureshiftdetector import FeatureShiftDetector
from fsd_models import GaussianDensity, Knn
from fsd_utils import (
    create_graphical_model,
    get_confusion_tensor,
    get_detection_metrics,
    get_localization_metrics,
    marginal_attack,
    plot_confusion_matrix,
    sim_copula_data,
)

sys.path.append("..")

from shift_utils import *

In [ ]:
SHIFT_EXPERIMENT = input("Select experiment: ")
OUTCOME = input("Select outcome variable: ")
if SHIFT_EXPERIMENT != "hosp_type":
    HOSPITAL = [input("Select hospital: ")]
NA_CUTOFF = 0.60

if SHIFT_EXPERIMENT == "covid":
    SHIFT_BS = ["pre-covid", "covid"]

if SHIFT_EXPERIMENT == "seasonal":
    SHIFT_BS = ["summer", "winter", "seasonal"]

(
    (X_train, y_train),
    (X_val, y_val),
    (X_test, y_test),
    feats,
    orig_dims,
) = import_dataset_hospital(
    SHIFT_EXPERIMENT, OUTCOME, HOSPITAL, NA_CUTOFF, shuffle=True
)

In [ ]:
# # Global Experiment Parameters
n_samples = 100  # The number of samples in p, q (thus n_samples_total = n_samples*2)
n_bootstrap_runs = 50
n_conditional_expectation = 30
n_inner_expectation = n_conditional_expectation
alpha = 0.05  # Significance level
data_family = "Copula"
a = 0.5
b = 0.5
rng = np.random.RandomState(42)
torch.manual_seed(rng.randint(1000))
method_list = [
    "score-method"
]  # we do not take the deep method into account with the simple boot.
# dataset_list = ['Energy', 'Gas', 'COVID']
dataset_list = ["COVID"]
t_split_interval = 50
n_comp_sensors_list = [1]
window_size_list = [i * 100 for i in range(0, 11)]
n_comp_sensors = 1

In [ ]:
n_trials = int(np.ceil((X_train.shape[0] - 2 * n_samples) / t_split_interval))
n_dim = X_train.shape[1]
sqrtn = int(np.floor(np.sqrt(n_dim)))
n_dataset_samples = X_train[n_samples:].shape[
    0
]  # to account for taking out n_samples for reference dist, p
rng = np.random.RandomState(42)
torch.manual_seed(rng.randint(1000))
print(n_trials)

In [ ]:
dataset_name = "gemini"
for method in method_list:
    for shuffle_data_set in [False, True]:
        # Experiment Switches
        if shuffle_data_set:
            shuffle_string = "time axis shuffled"
            experiment_name = f"time-boot-{method}-time-axis-shuffled-on-{dataset_name}"
        else:
            shuffle_string = "time axis unshuffled"
            experiment_name = (
                f"time-boot-{method}-time-axis-unshuffled-on-{dataset_name}"
            )
        print()
        print(
            f"Starting {method} on {dataset_name} dataset with {shuffle_string} and simple boot"
        )

        n_trials = int(np.ceil((X_train.shape[0] - 2 * n_samples) / t_split_interval))
        n_dim = X_train.shape[1]
        sqrtn = int(np.floor(np.sqrt(n_dim)))
        n_dataset_samples = X_train[n_samples:].shape[
            0
        ]  # to account for taking out n_samples for reference dist, p

        ## Attack testing  ##
        rng = np.random.RandomState(42)
        torch.manual_seed(rng.randint(1000))

        time_list = np.zeros(n_trials)
        global_truth = np.zeros(n_trials)
        detection = np.zeros(n_trials)
        detection_results = np.zeros(shape=(n_dim, n_trials, 3))
        j_attack = rng.choice(np.arange(n_dim), replace=True, size=n_trials)
        for idx, feature in enumerate(j_attack[: int(n_trials / 2)]):
            detection_results[feature, idx, 1] = 1  # recording where attacks happen
            global_truth[idx] = 1

        exception_occured = 0
        exception_vector = np.full(shape=(n_trials), fill_value=False)
        for test_idx, split_idx in enumerate(
            range(0, X_train.shape[0] - 2 * n_samples, t_split_interval)
        ):
            start = time()
            test_idx = int(test_idx)
            split_idx = int(split_idx)
            slice1 = split_idx
            slice2 = split_idx + 2 * n_samples
            #     try:
            pq = X_train[slice1:slice2]  # Two sets of samples
            pq = transform_data(
                pq, do_diff=do_diff, do_power_transform=do_power_transform
            )
            p = pq[:n_samples]
            q = pq[n_samples : n_samples * 2].copy()

            if np.any(detection_results[:, test_idx, 1] == 1):  # attack!
                attacked_features = j_attack[test_idx]
                q[:, attacked_features] = rng.permutation(
                    q[:, attacked_features]
                )  # permutes q

            # Bootstrap every time
            fsd = FeatureShiftDetection(
                p,
                q,
                rng=rng,
                samples_generator=np.nan,
                detection_method=method,
                n_bootstrap_runs=n_bootstrap_runs,
                n_conditional_expectation=n_conditional_expectation,
                n_attacks=np.nan,
                alpha=alpha,
                j_attack=np.nan,
                attack_testing=False,
            )
            bonferroni_threshold_vector = fsd.bonferroni_threshold_vector
            threshold_vector = fsd.threshold_vector
            bootstrap_score_means_vector = fsd.bootstrap_distribution.mean(axis=0)
            bootstrap_score_std_vector = (
                np.std(fsd.bootstrap_distribution, axis=0) + 1e-5
            )

            # now check after getting new threshold
            score_vector = np.array(fsd.get_score(p, q))
            detection_results[:, test_idx, 0] = score_vector
            # predicting attack
            if np.any(score_vector >= bonferroni_threshold_vector):
                detection[test_idx] = 1
                normalized_score_vector = (
                    score_vector - bootstrap_score_means_vector
                ) / bootstrap_score_std_vector
                attacked_features = normalized_score_vector.argsort()[-1]
                detection_results[attacked_features, test_idx, 2] = 1
            time_list[test_idx] = time() - start

        # Recording Attack Results
        confusion_tensor = np.zeros(shape=(n_dim, 2, 2))
        for feature_idx, feature_results in enumerate(detection_results):
            confusion_tensor[feature_idx] = sklearn_confusion_matrix(
                feature_results[:, 1], feature_results[:, 2], labels=[0, 1]
            )

        # overall detection confusion matrix
        global_detection_confusion_matrix = sklearn_confusion_matrix(
            global_truth, detection, labels=[0, 1]
        )

        full_tn, full_fp, full_fn, full_tp = confusion_tensor.sum(axis=0).flatten()
        micro_precision = full_tp / (full_tp + full_fp)
        micro_recall = full_tp / (full_tp + full_fn)

        if shuffle_data_set:
            print("Time axis shuffled")
        else:
            print("Time axis unshuffled")

        tn, fp, fn, tp = global_detection_confusion_matrix.flatten()
        detection_precision = tp / (tp + fp)
        detection_recall = tp / (tp + fn)

        print("Results for: ", experiment_name)
        print(f"Precision: {detection_precision * 100:.2f}%")
        print(f"Recall: {detection_recall * 100:.2f}%")

        print(f"Micro-precision: {micro_precision * 100:.2f}%")
        print(f"Micro-recall: {micro_recall * 100:.2f}%")

        print(f"Avg time per test: {time_list.mean():.2f} sec")
        print(f"Total time: {time_list.sum():.2f} sec")

        # Saving Score Distributions
        results_dict = {
            "detection_results": detection_results,
            "global_confusion_matrix": global_detection_confusion_matrix,
            "confusion_tensor": confusion_tensor,
            "times": time_list,
        }
        experiment_save_name = experiment_name + "-results_dict.p"
        pickle.dump(
            results_dict,
            open(path.join("..", "..", "results", experiment_save_name), "wb"),
        )
print(f'Experiment completed at {strftime("%a, %d %b %Y %I:%M%p", localtime())}')